In [1]:
# # creating a large table
# import sqlite3 as sql

# conn = sql.connect('test.db')
# c = conn.cursor()
# c.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT, age INTEGER, address TEXT, salary REAL)')
# # 100000000 4.7G (4m 41s)
# for i in range(100000):
# 	c.execute('INSERT INTO test VALUES (?, ?, ?, ?, ?)', (i, 'name'+str(i), i, 'address'+str(i), i*1000))
# conn.commit()
# conn.close()



KeyboardInterrupt



In [2]:
from bs4 import BeautifulSoup
htmlFile = open("course_directory_winter.html","r",encoding="utf8")
html = htmlFile.read()


In [3]:
# parsing the html 
# dumping the data into a txt file

soup = BeautifulSoup(html, 'html.parser')
soup.prettify()
tds = soup.find_all('td')
for td in tds:
	if td.text.strip() !="\n":
		textFile.write(td.get_text(separator=",")+"\n")




In [1]:
textFile = open("course_directory_winter.txt","r",encoding="utf8")
lines = textFile.readlines()



In [2]:
# first three letters of a course code are characters and the rest are numbers
def isCourseCode(string):
	string = string.replace(" ","")
	if len(string) == 6 and string[:3].isalpha() and string[3:].isdigit():
		return True
	
	return False

In [3]:
textFile = open("course_directory_winter.txt","r",encoding="utf8")
courses = []

for line in lines:
	if isCourseCode(line.split(",")[0]):
		courses.append(line.split(",")[0])
# monsoon 329/331 course found
# winter 283/282
print(len(courses), "Course found!")
# print(courses)


283 Course found!


In [4]:
# jsondict= []
courseArray=[]
course = ""
flag = True
for line in lines:
	line = line.replace("\t", "")
	if isCourseCode(line.split(",")[0]):
		# courseArray.append(line.split(",")[0])
		courseArray.append(course)
		course= ""
	course+=line
# print(courseArray)





In [5]:
courseArray = list(filter(lambda a: a.replace(" ","")!="",courseArray))
def detectType(s:str):
	Days = ["Mon", "Tue","Wed","Thu","Fri","Sat","Sun"]
	if s == "Section":
		return "Section"
	elif s in Days:
		return "Day"
	elif "-" in s:
		return "Date"
	elif ":" in s:
		return "Time"
	elif s == "to":
		return "To"
	elif s.isdigit():
		return "Section No."
	else:
		return "Useless"

In [8]:
import re
lines = []
Code = ''
Level = ''
Name = ''
isBiSem = ''
Credits = ''
Faculties = ''
Semester = ''
Prerequisite = ''
Description = ''
Sections = {}
SectionDict = {}
DayArray = [[],[]]
TimeArray = []
DateArray = []
activeSection = ''
activeDay = ''

oneCourseJson = {};
finalJson = []
for course in courseArray:
	Code = ''
	Level = ''
	Name = ''
	isBiSem = ''
	Credits = ''
	Faculties = ''
	Semester = ''
	Prerequisite = ''
	Description = ''
	Sections = {}
	SectionDict = {}
	DayArray = [[],[]]
	TimeArray = []
	DateArray = []
	activeSection = ''
	activeDay = ''
	lines = course.split("\n")
	lines = list(filter(lambda a: a.replace(" ","")!="",    lines))

	# print(lines)
	Code = lines[0].split(",")[0]
	Level = lines[0].split(",")[1]
	Level = Level.strip(" ").strip("[").strip("]")
	Name = lines[1]
	Credits = lines[2]
	Faculties = lines[3].split(",")
	Semester = lines[4]
	if Faculties[0] == "Not added":
		continue
	if lines[5].split(",")[0] == "PREQ_OR":
		Prerequisite = lines[5].split(",")[1]
		n = 5

		
	else:
		Prerequisite = None
		n = 4
	Description = lines[n+1]
	for i in lines[n+2:]:
		i = i.strip(",")
		i = i.replace(" ",",")
		i = i.split(",")

		i = list(filter(lambda a: a!="Quarter]",i))
		i = list(filter(lambda a: a!="[First",i))
		i = list(filter(lambda a: a!="[Second",i))
		i = list(filter(lambda a: len(a )!=0,i))
		i = list(map(lambda a: a.replace("[",""),i))
		i = list(map(lambda a: a.replace("]",""),i))
		i = list(map(lambda a: a.strip(","),i))
		# print(i)
		try: 
			for j in i:
				typeOfJ = detectType(j)
				# print(j,typeOfJ)
				if typeOfJ == "Section No.":
					activeSection = j
					Sections[j] = {}
				
				elif typeOfJ == "Day":
					activeDay = j
					if Sections[activeSection].get(activeDay) == None:
						Sections[activeSection][activeDay] = [[],[]]
					# Sections[activeSection][activeDay] = [[],[]]
				elif typeOfJ == "Time":
					if Sections[activeSection][activeDay][0].count(j) == 0:
						Sections[activeSection][activeDay][0].append(j)
					# Sections[activeSection][activeDay][0].append(j)
				elif typeOfJ == "Date":
					if Sections[activeSection][activeDay][1].count(j) == 0:
						Sections[activeSection][activeDay][1].append(j)
				elif typeOfJ == "To":
					pass
				elif typeOfJ == "Useless":
					pass
		except:
			pass
		   
	# print(Sections)

	oneCourseJson = {
		"Code":Code,
		"Level":Level,
		"Name":Name,
		"Credits":Credits,
		"Faculties":Faculties,
		"Semester":Semester,
		"Prerequisite":Prerequisite,
		"Description":Description,
		"Sections":Sections
	}
	finalJson.append(oneCourseJson)

import json
with open('winter.json', 'w') as outfile:
	json.dump(finalJson, outfile, indent=4)



	


	# print(lines[5], lines[6],12)
	# print(Code, Level, Name, Credits, Faculties,Semester, Prerequisite, Description)